In [99]:
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [100]:
titanic_data = pd.read_csv('C:/Users/taham/Desktop/7th semester/AI by Sir Ayyaz/Assignments/Titanic_data_Set.csv')                                                             #load dataset

In [101]:
X = (titanic_data[['PassengerId','Sex','Pclass','Fare','Age','SibSp','Parch']])
Y = (titanic_data[['Survived']])
X.shape

(891, 7)

In [89]:
Y.shape

(891, 1)

In [102]:
cols_to_Normal = ['PassengerId','Sex','Pclass','Fare','Age','SibSp','Parch']

X= X[cols_to_Normal]
X.head(10)

,PassengerId,Sex,Pclass,Fare,Age,SibSp,Parch
0,1,1,3,7.2500,22.0,1,0
1,2,2,1,71.2833,38.0,1,0
2,3,2,3,7.9250,26.0,0,0
3,4,2,1,53.1000,35.0,1,0
4,5,1,3,8.0500,35.0,0,0
5,6,1,3,8.4583,28.0,0,0
6,7,1,1,51.8625,54.0,0,0
7,8,1,3,21.0750,2.0,3,1
8,9,2,3,11.1333,27.0,0,2
9,10,2,2,30.0708,14.0,1,0


In [93]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.2)  

In [103]:
ls = [13, 5, 5, 1]                                                       #set layer sizes, do not change the size of the first and last layer 
iterations = 1000                                                                  #set number of iterations over the training set(also known as epochs in batch gradient descent context)
lr = 0.03  

In [104]:
def relu(z):
    a = np.maximum(0,z)
    return a

In [64]:
def par_init(ls):
    par = {}
    for i in range(1, len(ls)):
        par['W' + str(i)] = np.random.randn(ls[i], ls[i-1])*0.01
        par['B' + str(i)] = np.random.randn(ls[i],1)*0.01
    return par

In [65]:
def frwdPropagation(X_train, par):
    layers = len(par)//2
    values = {}
    for i in range(1, layers+1):
        if i==1:
            values['Z' + str(i)] = np.dot(par['W' + str(i)], X_train) + par['B' + str(i)]
            values['A' + str(i)] = relu(values['Z' + str(i)])
        else:
            values['Z' + str(i)] = np.dot(par['W' + str(i)], values['A' + str(i-1)]) + par['B' + str(i)]
            if i==layers:
                values['A' + str(i)] = values['Z' + str(i)]
            else:
                values['A' + str(i)] = relu(values['Z' + str(i)])
    return values


In [105]:
def cost(values, Y_train):
    layers = len(values)//2
    Y_pred = values['A' + str(layers)]
    cost = 1/(2*len(Y_train)) * np.sum(np.square(Y_pred - Y_train))
    return cost

In [106]:
def bcwrdPropagation(par, values, X_train, Y_train):
    layers = len(par)//2
    m = len(Y_train)
    grads = {}
    for i in range(layers,0,-1):
        if i==layers:
            dA = 1/m * (values['A' + str(i)] - Y_train)
            dZ = dA
        else:
            dA = np.dot(par['W' + str(i+1)].T, dZ)
            dZ = np.multiply(dA, np.where(values['A' + str(i)]>=0, 1, 0))
        if i==1:
            grads['W' + str(i)] = 1/m * np.dot(dZ, X_train.T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        else:
            grads['W' + str(i)] = 1/m * np.dot(dZ,values['A' + str(i-1)].T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return grads

In [67]:
def update_par(par, grads, lr):
    layers = len(par)//2
    par_updated = {}
    for i in range(1,layers+1):
        par_updated['W' + str(i)] = par['W' + str(i)] - lr * grads['W' + str(i)]
        par_updated['B' + str(i)] = par['B' + str(i)] - lr * grads['B' + str(i)]
    return par_updated

In [68]:
def model(X_train, Y_train, ls, iterations, lr):
    par = par_init(ls)
    for i in range(iterations):
        values = frwdPropagation(X_train.T, par)
        cost = cost(values, Y_train.T)
        grads = bcwrdPropagation(par, values,X_train.T, Y_train.T)
        par = update_par(par, grads, lr)
        print('Cost at iteration ' + str(i+1) + ' = ' + str(cost) + '\n')
    return par

In [107]:
def compute_accuracy(X_train, X_test, Y_train, Y_test, par):
    values_train = frwdPropagation(X_train.T, par)
    values_test = frwdPropagation(X_test.T, par)
    train_acc = np.sqrt(mean_squared_error(Y_train, values_train['A' + str(len(ls)-1)].T))
    test_acc = np.sqrt(mean_squared_error(Y_test, values_test['A' + str(len(ls)-1)].T))
    return train_acc, test_acc

In [108]:
def predict(X, par):
    values = frwdPropagation(X.T, par)
    predictions = values['A' + str(len(values)//2)].T
    return predictions

In [109]:
p = model(X_train, Y_train, ls, iterations, lr)           
train_acc, test_acc = compute_accuracy(X_train, X_test, Y_train, Y_test, par)  
print('Root Mean Squared Error on Training Data = ' + str(train_acc))
print('Root Mean Squared Error on Test Data = ' + str(test_acc))

ValueError: shapes (5,13) and (7,712) not aligned: 13 (dim 1) != 7 (dim 0)